In [ ]:
#!/usr/bin/python2

import os
import time
import json
import serial
import subprocess
#from phant import Phant
import paho.mqtt.client as mqtt
#from upload_phant import upload_phant
import sys

"""
This is the code for the latest fiber test.
The peripherals include:
    1*scales
    3*moisture sensors
    3*suction sensors
    1*webcam
    1*flashlight
"""
#------------------- Constants and Ports Information---------------------------

SCREEN_DISPLAY = True
SAVE_TO_FILE = True
PUBLISH_TO_THINGSBOARD = False
DELIMITER = ','
SLEEP_TIME_SECONDS = 60*30 # s
SERIAL_PORT = '/dev/ttyS0' # datalogger version 2 uses ttyS0
SERIAL_BAUD = 9600
#SERIAL_PORT = '/dev/serial/by-path/platform-3f980000.usb-usb-0:1.4:1.0' # datalogger version 1 uses ttyACM0
# PORT_SCALE = '/dev/serial/by-path/platform-20980000.usb-usb-0:1:1.0-port0' #the USB port connected to the scale
# PORT_SCALE = '/dev/serial/by-id/usb-FTDI_USB__-__Serial-if00-port0'
#HEAT_TIME = 24000

#---------------------- Configuration -------------------------

file_name = 'yarwun_column.csv'

field_name = ['scale',
            'ec5',
            '555 K=0.05 R=21k #3',
            '555 K=0.05 R=21k #1',
            '555 K=0.05 R=21k #2', 
            '555 K=0.05 R=20k',
            '555 K=0.05 R=21k #4',
            '555 K=0.05 R=21k #5',
            'temp',
            'humidity']

data_collected = dict((el,0.0) for el in field_name)

#---------------------------- Initiation --------------------------------------
if (PUBLISH_TO_THINGSBOARD):
    with open('/home/pi/pyduino/credential/testbench_basin.json') as f: 
        credential = json.load(f)

    try:
        client = mqtt.Client()
        client.username_pw_set(credential['access_token'])
        client.connect(credential['thingsboard_host'], 1883, 60)
        client.loop_start()
    except Exception:
        print("Failed to connect to thingsboard")
        # time.sleep(30)
 
try:    
    # while True:
    ard = serial.Serial(port=SERIAL_PORT, baudrate=SERIAL_BAUD, timeout=60)
    time.sleep(5)
    
    time_now = time.time()
    time_now_local = time.localtime(time_now)
    seconds_since_epoch = int(round(time_now * 1000))
    time_now_local_str = time.strftime("%Y-%m-%d %H:%M:%S", time_now_local)

    # Open file for appending
    if SAVE_TO_FILE:
        if not os.path.isfile(file_name):
            fid = open(file_name, 'a', 0) # Open file for appending
            fid.write("Timestamp,Temp (C),Humidity (%),Analog1,Analog2,Analog3,Analog4,Analog5,Analog6,Analog7,Analog8,Analog9,Analog10\r\n") # Allocate column names
        fid = open(file_name, 'a', 0) # Open file for appending
        # fid.write(time_now_local_str + '\r\n')
        fid.write(time_now_local_str)   # For standard csv format

    if SCREEN_DISPLAY:
        print(time_now_local_str)

    #---------------------------------Temp & Hum DHT22-----------------------------
    # DHT22 temp and humidity onboard
    try:
        ard.write("dht22,54,power,2,points,3,dummies,1,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read = msg.split(',')[0:-1]
        dht22_temp = float(current_read[-2])
        dht22_hum = float(current_read[-1])
        data_collected['temp'] = dht22_temp
        data_collected['humidity'] = dht22_hum
        if SCREEN_DISPLAY:
            print("Ambient Temp (C)" + ": " + str(dht22_temp))
            print("Ambient Humdity (%): " + str(dht22_hum))

        if SAVE_TO_FILE:
            fid.write(DELIMITER + str(dht22_temp))
            fid.write(DELIMITER + str(dht22_hum))
            # fid.write(time_now_local_str + DELIMITER + 'Temp' + DELIMITER + "C" + DELIMITER+DELIMITER + str(dht22_temp) + '\r\n')
            # fid.write(time_now_local_str + DELIMITER + 'Humidity' + DELIMITER + "%" + DELIMITER+DELIMITER + str(dht22_hum) + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("DHT22 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 1--------------
    # MS555 K = 0.05, R = 20k
    try:
        field_name_id = 5
        ard.write("analog,14,power,44,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=20k'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 K=0.05 R=20k reading failed, error: {0}".format(sys.exc_info()[0]))
        

    #----------------------------Moisture sensor 2---------------------------------
    # MS555 k=0.05 R=21k #3
    try:
        field_name_id = 2
        ard.write("analog,13,power,43,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read = msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #3'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #3 reading failed, error: {0}".format(sys.exc_info()[0]))


    #----------------------------Moisture sensor 3---------------------------------
    # MS555 k=0.05 R=21k #2
    try:
        field_name_id = 4
        ard.write("analog,11,power,41,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #2'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #2 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 4--------------------------------
    # MS555 k=0.05 R=21k #1
    try:
        field_name_id = 3
        ard.write("analog,10,power,40,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #1'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #1 reading failed, error: {0}".format(sys.exc_info()[0]))

    #--------------------------Moisture Sensor 5-------------------------------
    # EC5
    try:
        field_name_id = 1
        ard.write("analog,12,power,42,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['ec5'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("EC5 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 6--------------------------------
    # MS555 k=0.05 R=21k #4
    try:
        field_name_id = 6
        ard.write("analog,9,power,39,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #4'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #4 reading failed, error: {0}".format(sys.exc_info()[0]))

    #---------------------------Moisture Sensor 7--------------------------------
    # MS555 k=0.05 R=21k #5
    try:
        field_name_id = 7
        ard.write("analog,8,power,38,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #5'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #4 reading failed, error: {0}".format(sys.exc_info()[0]))


    #----------------------------Upload data -----------------------------------    
    if (ard.isOpen()):
        ard.close()

    if (PUBLISH_TO_THINGSBOARD):
        json_data = {"ts":seconds_since_epoch, "values": data_collected}
        # json_data = data_collected
        try:
            # Result is in tuple (rc, mid) of MQTTMessageInfo class
            publish_result = client.publish('v1/devices/me/telemetry', payload=json.dumps(json_data), qos=1)
            # publish_result.wait_for_publish(timeout=1)
            # print(publish_result.is_published())
            # if (not publish_result.is_published()):
            if (publish_result.rc != mqtt.MQTT_ERR_SUCCESS):
                if SCREEN_DISPLAY:
                    print(mqtt.error_string(publish_result.rc))
                json_filename = "tsqueue_testbench_basin.json"
                listObj = []
                # Read json file
                if (os.path.isfile(json_filename)):
                    with open(json_filename) as json_file:
                        listObj = json.load(json_file)
                # Verify existing list
                # print(listObj)
                # print(type(listObj))
                # print(json.dumps(listObj))
                # print(json.dumps(listObj[0]))
                listObj.append(json_data)
                with open(json_filename, 'w') as json_file:
                    json.dump(listObj, json_file, indent=4, separators=(",",": "))
        except (ValueError, RuntimeError) as error:
            if SCREEN_DISPLAY:
                print(error)
        finally:
            if SCREEN_DISPLAY:
                print(publish_result)
                print(json_data)

    if SCREEN_DISPLAY:
        print("")

    if SAVE_TO_FILE:
        fid.write("\r\n")
        if (fid.closed == False):
            fid.close()

    # print("begin to sleep")
    # time.sleep(SLEEP_TIME_SECONDS) # sleep to the next loop
       

except KeyboardInterrupt:
    pass

finally:
    if (SAVE_TO_FILE and (fid.closed == False)):
        fid.close()
    if (ard.isOpen()):
        ard.close()
    if (PUBLISH_TO_THINGSBOARD):
        client.loop_stop()
        client.disconnect()
